# Erklärung Hough Transformation

Dieser Abschnitt erklärt die meisten Methoden der Klasse HoughTransformation. Weggelassen wurden Methoden, welche nur für die
Entwicklung benötigt wurden und im Produktivcode nicht aufgerufen werden (Debugging Methoden).

### Disclaimer
Der Code ist nicht lauffähig, dafür existieren die Python Skripte. Das Notebook stellt lediglich eine Dokumentation dar.

### Verbesserungen, welche an Calib vorgenommen wurden
Versuch ein Polynom durch die Punkte zu legen und nicht nur eine Linie und dieses Polynom durch einen möglichen
Fix-Punkt zu verbessern.

In den Ursprünglichen Ansätzen wurde der Mittelpunkt aller Werte eines Linienbereichs gebildet und dadurch eine Linie
gelegt, die keine Krümmung hat. So kann zwar eine Linie mit wenig Krümmung im nahen Bereich am Fahrzeug die Spur
erkennen aber nicht in der Ferne (1. Abbildung). Um das zu Verbessern wurde ein Ansatz entwickelt, um durch die Punkte eine Polynom
höheren Grades zu legen. Dadurch kann eine Krümmung auch in der Ferne abgebildet werden. Doch damit kommt das nächste
Problem. Es werden nicht immer genug Linien im Bild gefunden um ein perfektes Polynom durch die Punkte der Linien zu
legen. Es entstehen Polynome mit starker Krümmung (2. Abbildung). Um dieses Problem zu Lösen, wurden Fixpunkte konfiguriert, die in die
Berechnung des Polynoms einfließen, wenn der unterste gefundene Punkt über einem bestimmten Schwellwert ist. Dadurch
kann das Polynom an die Spur angepasst werden  (3. Abbildung).

##### Bild nur Linie nicht Polynom höheren Grads
<img src="Images/hough_without_polynome.png" width="500">

##### Bild ohne Fix-Punkt und 2. Grad:
<img src="Images/hough_without_fixpoint.png" width="500">

##### Bild mit Fix-Punkt und 2. Grad (Finale Lösung)
<img src="Images/hough_draw_polyline.png" width="500">

### Import aller spezifischen Bibliotheken und Module
Zu beachten ist die Bibliothek calib importiert als cal für die Kamerakalibrierung und preprocess importiert als pre für
wichtige Vorverarbeitungsmethoden.

In [ ]:
import cv2 as cv
import numpy as np
import time
import json
import os

import calib as cal
import preprocess as pre

### Klassendeklaration und init Methode
Hier wird die Klasse initialisiert. Dafür wird ein Objekt für die Klasse Prerocess erstellt und das Flag für das laden
der Konfiguration auf False gesetzt.

In [ ]:
class HoughTransformation():

    def __init__(self, debug = False) -> None:
        """Constructor of the class HoughTransformation

        Args:
            debug (bool, optional): Debug Mode on or off. Defaults to False.
        """
        self.pre = pre.Preprocess()
        self.debug = debug
        self.loaded = False

## Load config
In dieser Methode wird die abgespeicherte Konfiguration geladen. Dafür muss der Dateipfad übergeben werden und alle
benötigten Keys enthalten. Das verwendete Datenformat ist hier JavaScript Object Notation (JSON). Ist ein benötigter Key
nicht vorhanden, wird eine Fehlernachricht zurückgegeben. Wenn alle Keys enthalten sind, dann werden sie in Variablen
gespeichert. Bei erfolgreicher Konfiguration wird das Flag für das Laden der Konfiguration auf True gesetzt.

In [ ]:
def load_config(self, path):
    if not os.path.exists(path):
        return print('File '+ path +' not found')
    
    with open(path, 'r') as f:
        config = json.load(f)

    if not config:
        return 'Error: Config file is empty'
    if not 'HOUGH' in config.keys():
        return 'Error: HOUGH is missing'
    if not 'CANNY_LOWER' in config['HOUGH'].keys():
        return 'Error: CANNY_LOWER is missing'
    if not 'CANNY_UPPER' in config['HOUGH'].keys():
        return 'Error: CANNY_UPPER is missing'
    if not 'ROI' in config['HOUGH'].keys():
        return 'Error: ROI is missing'
    if not 'RHO' in config['HOUGH'].keys():
        return 'Error: RHO is missing'
    if not 'THETA' in config['HOUGH'].keys():
        return 'Error: THETA is missing'
    if not 'THRESHOLD' in config['HOUGH'].keys():
        return 'Error: THRESHOLD is missing'
    if not 'MIN_LINE_LENGTH' in config['HOUGH'].keys():
        return 'Error: MIN_LINE_LENGTH is missing'
    if not 'MAX_LINE_GAP' in config['HOUGH'].keys():
        return 'Error: MAX_LINE_GAP is missing'
    if not 'LEFT_FIX' in config['HOUGH'].keys():
        return 'Error: LEFT_FIX is missing'
    if not 'RIGHT_FIX' in config['HOUGH'].keys():
        return 'Error: RIGHT_FIX is missing'
    if not 'BORDER_LEFT' in config['HOUGH'].keys():
        return 'Error: BORDER_LEFT is missing'
    if not 'BORDER_RIGHT' in config['HOUGH'].keys():
        return 'Error: BORDER_RIGHT is missing'
    if not 'POLY_HEIGHT' in config['HOUGH'].keys():
        return 'Error: POLY_HEIGHT is missing'
    if not 'MIN_COLOR' in config['HOUGH'].keys():
        return 'Error: MIN_COLOR is missing'
    if not 'MAX_COLOR' in config['HOUGH'].keys():
        return 'Error: MAX_COLOR is missing'
    if not 'HIT_X_LEFT' in config['HOUGH'].keys():
        return 'Error: HIT_X_LEFT is missing'
    if not 'HIT_Y_LEFT' in config['HOUGH'].keys():
        return 'Error: HIT_Y_LEFT is missing'
    if not 'HIT_X_RIGHT' in config['HOUGH'].keys():
        return 'Error: HIT_X_RIGHT is missing'
    if not 'HIT_Y_RIGHT' in config['HOUGH'].keys():
        return 'Error: HIT_Y_RIGHT is missing'
    if not 'HIT_X_MIDDLE_LEFT' in config['HOUGH'].keys():
        return 'Error: HIT_X_MIDDLE_LEFT is missing'
    if not 'HIT_X_MIDDLE_RIGHT' in config['HOUGH'].keys():
        return 'Error: HIT_X_MIDDLE_RIGHT is missing'
    if not 'HIT_Y_MIDDLE' in config['HOUGH'].keys():
        return 'Error: HIT_Y_MIDDLE is missing'
    
    self.canny_lower = config['HOUGH']['CANNY_LOWER']
    self.canny_upper = config['HOUGH']['CANNY_UPPER']
    self.roi = config['HOUGH']['ROI']
    self.roi2 = config['HOUGH']['ROI2'] if 'ROI2' in config['HOUGH'].keys() else None
    self.rho = config['HOUGH']['RHO']
    self.theta = config['HOUGH']['THETA']
    self.threshold = config['HOUGH']['THRESHOLD']
    self.min_line_length = config['HOUGH']['MIN_LINE_LENGTH']
    self.max_line_gap = config['HOUGH']['MAX_LINE_GAP']
    self.left_fix = config['HOUGH']['LEFT_FIX']
    self.right_fix = config['HOUGH']['RIGHT_FIX']
    self.border_left = config['HOUGH']['BORDER_LEFT']
    self.border_right = config['HOUGH']['BORDER_RIGHT']
    self.poly_height = config['HOUGH']['POLY_HEIGHT']
    self._min_color = config['HOUGH']['MIN_COLOR']
    self._max_color = config['HOUGH']['MAX_COLOR']
    self._hit_x_left = config['HOUGH']['HIT_X_LEFT']
    self._hit_y_left = config['HOUGH']['HIT_Y_LEFT']
    self._hit_x_right = config['HOUGH']['HIT_X_RIGHT']
    self._hit_y_right = config['HOUGH']['HIT_Y_RIGHT']
    self._hit_x_middle_left = config['HOUGH']['HIT_X_MIDDLE_LEFT']
    self._hit_x_middle_right = config['HOUGH']['HIT_X_MIDDLE_RIGHT']
    self._hit_y_middle = config['HOUGH']['HIT_Y_MIDDLE']
    
    
    self.loaded = True
    return None

## Execute
Diese Methode führt die Hough Transformation komplett durch und gibt das verarbeitete Bild mit eingezeichneten Polynomen
zurück. Dafür wird das Bild zuerst vorverarbeitet. Danach werden mit `self._getHoughLines` die Linien im Bild über die
Hough Transformation gefunden. Danach werden die Punkte, die die Linien bilden, in linke und rechte Spur gruppiert.
Wurden Punkte für eine linke Spur bzw. rechte Spur gefunden, wird versucht Polynome zu finden, die durch alle Punkte
gehen. Für diese Polynome werden die Punkte bis zu einem definierten Horizont berechnet und auf ihre Plausibilität
geprüft. Ist alles in Ordnung, werden die Polynome im letzten Schritt in das Bild eingezeichnet und das fertig
verarbeitete Bild zurückgegeben.

In [ ]:
def execute(self, img):
    """Execute the Hough transformation with it pre-processing steps

    Args:
        img (Img/Frame): Is the frame of the video
        
    Returns:
        Image: The current frame
    """
    
    if not self.loaded:
        return False
    
    processed_img = self._preprocess(img)

    # Hough transformation
    lines = self._getHoughLines(processed_img)
    
    if self.debug: 
        img_debug = self._drawLines(img, lines)
        cv.imshow("Debug: Hugh", img_debug)
    
    # Group the lines into a left and a right group
    line_info = self._group_line_points(img, lines)
    
    # Map line infos
    left_x = line_info['LEFT_X']
    left_y = line_info['LEFT_Y']
    right_x = line_info['RIGHT_X']
    right_y = line_info['RIGHT_Y']
    
    left_line = None
    right_line = None
    
    if len(left_x) > 0 and len(left_y) > 0:
        left_line = self._get_polyLine_points(img, left_x, left_y, self.left_fix, self.border_left)

    if len(right_x) > 0 and len(right_y) > 0:
        right_line = self._get_polyLine_points(img, right_x , right_y, self.right_fix, self.border_right)
    
    # Check for crossing lines
    if left_line and right_line and any(left_line['FIT_X'] >= right_line['FIT_X']):
        return img
    
    if not left_line and not right_line:
        return img
    
    if left_line and self.check_plausibility(left_line, img.shape):
        processed_img = self._draw_poly_line_hugh(img, left_line, (255,0,0))
    if right_line and self.check_plausibility(right_line, img.shape):
        processed_img = self._draw_poly_line_hugh(img, right_line)
        
    return processed_img

## Preprocess
Hier wird das Bild für die Hough Transformation vorverarbeitet. Zuerst wird versucht die gelbe Linie aus dem Bild auf
Weiss zu ändern (mit `self.pre.map_color`), damit später der Kantenfilter die Kante besser erkennt. Danach wird das Bild
in Graustufen geändert und mit dem Gauss Filter ausgeglichen. Zum erkennen der Kanten wird jetzt der Canny Edge Filter
angewendet. Dadurch erhält man ein Schwarz-Weisses Bild in dem die Kanten mit einer Linie markiert sind. Im nächsten
Schritt wird mindestens eine Region of Interest festgelegt die den zu untersuchenden Bildbereich festlegt. In diesem
Projekt ein Viereck in der unteren Hälfte des Bilds. Dann kann optional eine zweite Region of Interest festgelegt
werden, die einen Bereich aus der ersten Region of Interest entfernt. Das entstandene Bild wird dann zurückgegeben
(siehe Bild).

<img src="Images/roi_hough.png" width="500" >

In [ ]:
def _preprocess(self, img):
    """Preprocessing steps for the hough transformation

    Args:
        img (Img/Frame): Is the current frame of the video
    
    Returns:
        Image: The current frame
    """
    # Find the yellow line
    if self._min_color and self._max_color:
        img = self.pre.map_color(img, self._min_color, self._max_color)
        if self.debug:
            cv.imshow('yellow', img)
    
    # Convert to grayscale
    img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

    # Decrease noise via Gauss
    img = self.pre.gauss(img)
    
    # Apply Canny edge detection
    img = self.pre.canny(img, self.canny_lower, self.canny_upper)

    # Segmentation of the image
    img = self.pre.segmentation(img, self.roi)
    if self.roi2:
        img = self.pre.segmentation(img, self.roi2, True)
    if self.debug:
        cv.imshow("Debug: Segmentation", img)
    
    return img


## Get hough lines
Diese Methode findet Linien auf Basis der Hough Transformation. Als Ergebnis werden pro Linie zwei Punkte mit je X und Y
Koordinaten ausgegebene.

In [ ]:
def _getHoughLines(self, img):
    """Find the lines in the image via Hough transformation

    Args:
        img (Img/Frame): Current Frame

    Returns:
        Array: Coordinates of the lines
    """
    lines = cv.HoughLinesP(
        img,
        rho=self.rho,
        theta=np.pi / self.theta,
        threshold=self.threshold,
        lines=np.array([]),
        minLineLength=self.min_line_length,
        maxLineGap=self.max_line_gap
    )

    return lines

## Draw lines
Diese Methode dient zum Einzeichnen von Linien in das Bild. Die Linien müssen durch 2 Punkte definiert werden. Im
Projekt wird diese Methode nur zum Ausgeben der Hough Linien im Debug Modus verwendet (siehe Bild). Um das zu erreichen
wird jede einzelne Linie nacheinander mit `cv.line` in das Bild gezeichnet. Es besteht die Möglichkeit über die
Eingabeparameter die Farbe und Breite der Linien zu ändern.

<img src="Images/hough_draw_line.png" width="500">

Die in diesem Bild erkannten Linien sind, jeweils die Kanten der Spuren, weil das Bild zuvor mit Canny Edge bearbeitet
wurde und darauf die Hough Transformation angewendet wurde.

In [ ]:
def _drawLines(self, img, lines, color=[0,0,255], thickness=10):
    """Draws lines by given coordinates into an image

    Args:
        img (Image): Current Frame
        lines (Array): Coordinates of the lines
        color (list, optional): Color of the lines. Defaults to [0,0,255].
        thickness (int, optional): Thickness of the lines. Defaults to 10.

    Returns:
        Image: Image with the the lines
    """        
    # Make a copy of the original image.
    img = np.copy(img)
    
    # Create a blank image that matches the original in size.
    line_img = np.zeros(
        (
            img.shape[0],
            img.shape[1],
            3
        ),
        dtype=np.uint8,
    )

    # Check if any lines were detected
    if lines is not None:

        # Loop over all lines and draw them on the blank image.
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv.line(line_img, (x1, y1), (x2, y2), color, thickness)
    
    # Add the lines to the original image
    img = cv.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    
    # Return the modified image.
    return img


## Group line points
Diese Methode wird verwendet, um die einzelnen Punkte der gefundenen Linien in eine linke und rechte Spur zu gruppieren.
Dafür wird das Bild in zwei Hälften geteilt und ein Bereich Angegeben in dem kein Punkt sein darf um Outlier zu
verhindern, was das Ergebnis verbessern kann. Dabei wird in diesem Beispiel nur jeder Punkt als linke Spur betrachtet,
wenn der X-Wert kleiner als `mitte - 0.06 * mitte` ist und als rechte, wenn der X-Wert größer als `mitte + 0.06 *
mitte`. Es existiert also ein Bereich mit `0.12 * mitte` in dem keine Punkte erkannt werden. Außerdem werden die Punkte
nicht mehr zu ihrer Ursprünglichen Linie zugewiesen, sondern nur noch in linke Seite und rechte Seite. Das ist wichtig
für die spätere Zuweisung des Polynoms. Diese Zuweisungen werden dann in einem Dictionary zurückgegeben.

In [ ]:
def _group_line_points(self, img, lines):
        """Groups the given line coordinates for the left and right
        part of the image

        Args:
            img (Image): Current Frame
            lines (Array): Line coordinates
        """
         # Get the mid of the picture
        mid = img.shape[1]//2

        left_x = []
        left_y = []

        right_x = []
        right_y = []
        
        # Checks if there are any lines
        if lines is None:
            return {
                'LEFT_X': left_x,
                'LEFT_Y': left_y,
                'RIGHT_X': right_x,
                'RIGHT_Y': right_y,
            }

        factor = 0.06

        for line in lines:
            # Everything 10% left of the mid
            if line[0][0] <= mid - (mid * factor):
                left_x.append(line[0][0])
                left_y.append(line[0][1])
            elif line[0][0] >= mid + (mid * factor):
                right_x.append(line[0][0])
                right_y.append(line[0][1])

            if line[0][2] <= mid - (mid * factor):
                left_x.append(line[0][2])
                left_y.append(line[0][3])
            elif line[0][0] >= mid + (mid * factor):
                right_x.append(line[0][2])
                right_y.append(line[0][3])
        
        return {
            'LEFT_X': left_x,
            'LEFT_Y': left_y,
            'RIGHT_X': right_x,
            'RIGHT_Y': right_y,
        }

## Get polyline points
Hier werden die in `groupLinePoints` gruppierten Punkte verwendet, um ein passendes Polynom zu finden. Dafür wird über
`np.polyfit` ein Polynom 2. Grades gefunden. Danach werden die einzelnen Punkte die in das Bild fallen berechnet. Die
gefundenen X-Koordinaten bzw. Y-Koordinaten, werden dann zurückgegeben. Wichtig ist das ein Fix-Punkt angegeben werden
kann, weil bei der Hough Transformation meistens nur Punkte gefunden werden die nicht am unteren Bildrand sind. Dadurch
macht das Polynom ohne Fix-Punkt eine Kurve. Durch einen Fixpunkt kann das verhindert werden, aber dafür das Ergebnis
verfälscht werden. Um das zu vermeiden wird der Punkt nur zur Bestimmung verwendet, wenn der unterste gefundene Punkt
überhalb einer vorkonfigurierten Grenze ist. In den Ursprünglichen Ansätzen der Hough Transformation wurden keine
Polynome durch die Punkte gelegt, sondern nur der Durchschnitt der Hough Linien genommen und eine Line durchgezogen.
Dadurch konnte die Kurve aber nicht erkannt werden. Deswegen wurde hier diese Verbesserung durchgeführt.

##### Bild nur Linie nicht Polynom höheren Grads
<img src="Images/hough_without_polynome.png" width="500">

##### Bild ohne Fix-Punkt und 2. Grad
<img src="Images/hough_without_fixpoint.png" width="500">

##### Bild mit Fix-Punkt und 2. Grad
<img src="Images/hough_draw_polyline.png" width="500">

In [ ]:
def _get_polyLine_points(self, img, x, y, fix_point, border):
    """Generates the polygon fitting the coordinates

    Args:
        img (Image): Current frame
        x (List): x-coordinates of the points
        y (List): y-coordinates of the points
        fix_point (List): Coordinates of an additional fix points
        border (Int): Border of drawing the line (horizon)

    Returns:
        Dict: Info with the fitting x-coordinates (FIT_X) and
        y-coordinates (PLOT_Y)
    """
    # Add point of car if the nearest point is further away then the
    # provided value
    if y[np.argmax(y)] < border:
        x.append(fix_point[0])
        y.append(fix_point[1])

    #Generate poly lines
    poly = np.polyfit(y,x,2)

    # Generate the points
    plot_y = np.linspace(self.poly_height, img.shape[0] - 1, img.shape[0])
    fit_x = poly[0] * plot_y**2 + poly[1] * plot_y + poly[2]
    
    return {
        'FIT_X': fit_x,
        'PLOT_Y': plot_y,
    }    

## Check plausibility
Diese Method überprüft, ob das erzeugte Polynom plausibel ist. Dafür werden Bereiche definiert, in denen keine Punkt des
Polynoms liegen darf. In diesem Projekt sind das die linke und die rechte Untere Ecke. Liegt dort ein Punkt des
Polynoms, dann gab es entweder zu wenig Punkte zum Berechnen des Polynoms, die Punkte waren zu weit weg oder es gab
Störpunkte, die das Ergebnis verfälschen. Liegt kein Punkt in einem nicht plausiblen Bereich, dann wird `True`
zurückgegeben, ansonsten `False`.

In [ ]:
def check_plausibility(self, draw_info, img_shape) -> bool:
    y_values = draw_info['PLOT_Y']
    fit_x = draw_info['FIT_X']
    
    # Check for the hit boxes
    if any(fit_x[self._hit_y_left:] <= self._hit_x_left):
        return False
        
    if any(fit_x[self._hit_y_right:] >= img_shape[1] + self._hit_x_right):
        return False
    
    if any(fit_x[img_shape[0] + self._hit_y_middle:] >= self._hit_x_middle_left) and any(fit_x[img_shape[0] + self._hit_y_middle:] <= img_shape[1] + self._hit_x_middle_right):
        return False
    
    return True

## Draw poly line Hough
Diese Methode zeichnet das Polynom in das Ursprüngliche Bild ein. Dafür werden die Punkte des Polynoms genommen und per
`cv.polylines` in das Bild eingetragen. Über die Übergabe Parameter kann dann die Farbe der Linie eingestellt werden.

<img src="Images/hough_draw_polyline.png" width="500">

In [ ]:
def _draw_poly_line_hugh(self, img, draw_info, color = (0,0,255)):
    """Draw the polynomial in the image

    Args:
        img (Image): Current Frame
        draw_info (List): Coordinates of the points to draw
        color (tuple, optional): Color of the line. Defaults to (0,0,255).

    Returns:
        Image: Processed frame
    """
    # Unpack draw Info
    fit_x = draw_info['FIT_X']
    plot_y = draw_info['PLOT_Y']

    # Check whether data exist
    if len(fit_x) <= 0 and len(plot_y) <= 0:
        return img

    # Generate the points of the lane
    pts = np.array(np.transpose(np.vstack([fit_x, plot_y])))

    pts = pts.reshape((-1, 1, 2))

    # Draw the driving lane in the transformed image
    cv.polylines(img, np.int_([pts]), False, color, 4)

    return img